In [128]:
pip install afinn

  Stored in directory: C:\Users\Avi\AppData\Local\pip\Cache\wheels\b5\1c\de\428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import libraires
import spacy
import gensim
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim
from afinn import Afinn

In [17]:
#Load and View Dataset
df = pd.read_csv('test.csv')
df.head()

,tweet_id,tweet
0,7506,Audience Q: What prototyping tools do you use?...
1,7992,At SXSW? Send Your Best Photos &amp; Videos to...
2,247,@mention and here's a pic of you winning your...
3,7688,Google Marissa Mayer: mobile phone as a cursor...
4,3294,#SXSW Google maps is even cooler than I thought


In [18]:
#Check for null values
df.isnull().sum()

tweet_id    0
tweet       0
dtype: int64

In [19]:
#Drop null values
df = df.dropna()

In [20]:
#Convert tweets into list
tweets = df['tweet'].tolist()

In [21]:
#Initialize an afinn lexicon
afinn = Afinn()

In [22]:
#View the sentiment score for each hashtag
afinn_scores = [afinn.score(text) for text in tweets]

In [23]:
#Make a df of afinn scores
afinn_df = pd.DataFrame(afinn_scores)
afinn_df

,0
0,0.0
1,3.0
2,4.0
3,2.0
4,0.0
5,0.0
6,0.0
7,3.0
8,-2.0
9,5.0


In [24]:
afinn_df.to_csv('text_score_test.csv')

In [6]:
#View all hashtags in the provided tweets
hashtags = []
for tweet in tweets:
    hashtag = re.findall(r"#(\w+)", tweet)
    hashtags.append(hashtag)
    
flat_list = [item for sublist in hashtags for item in sublist]
flat_list = [x.lower() for x in flat_list]
flat_list = [x for x in flat_list if x != 'sxsw']

In [7]:
for i in range(0, len(hashtags)):
    for x in hashtags[i]:
        x.lower()

In [8]:
hashtags = [' '.join(x) for x in hashtags]

In [9]:
hashtags_df = pd.DataFrame(hashtags)
hashtags_df[0] = hashtags_df[0].apply(lambda x: x.lower())

In [10]:
hashtags_df.to_csv('hashtags_test.csv')

In [25]:
#Get unique list of hashtags
hashtags_unique = []
for x in flat_list:
    if x not in hashtags_unique:
        hashtags_unique.append(x)

In [11]:
#Initialize an afinn lexicon
afinn = Afinn()

In [12]:
#View the sentiment score for each hashtag
afinn_scores = [afinn.score(text) for text in hashtags]

In [13]:
#Make a df of afinn scores
afinn_df = pd.DataFrame(afinn_scores)
afinn_df['hashtags'] = hashtags

In [14]:
afinn_df.to_csv('afinn_score_test.csv')

In [12]:
#Important hashtags
afinn_df = afinn_df[afinn_df[0] != 0]

In [13]:
#Rename column
afinn_df = afinn_df.rename(columns = {0: 'Sentiment Score'})

In [14]:
#Number of hashtags which have a sentiment
print("The number of hashtags which have a sentiment are {}.".format(afinn_df.shape[0]))

The number of hashtags which have a sentiment are 52.


In [15]:
#View the important hashtags
afinn_df.head(10)

,Sentiment Score,hashtags
33,4.0,awesome
70,-2.0,fail
82,1.0,shared
111,4.0,winning
136,2.0,philanthropy
142,3.0,super
147,1.0,free
214,-2.0,fools
257,4.0,wow
262,-2.0,awkward


In [16]:
#Function to determine sentiment category
def sentiment_category(x):
    if x < 0:
        return 'negative'
    else:
        return 'positive'

In [17]:
#Apply function on df
afinn_df['sentiment category'] = afinn_df['Sentiment Score'].apply(lambda x: sentiment_category(x))

In [18]:
#View df
afinn_df

,Sentiment Score,hashtags,sentiment category
33,4.0,awesome,positive
70,-2.0,fail,negative
82,1.0,shared,positive
111,4.0,winning,positive
136,2.0,philanthropy,positive
142,3.0,super,positive
147,1.0,free,positive
214,-2.0,fools,negative
257,4.0,wow,positive
262,-2.0,awkward,negative


In [19]:
afinn_df.to_csv('hastag_scores.csv')